In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import ase
from ase.neighborlist import natural_cutoffs, NeighborList
from ase.io.trajectory import Trajectory
from ase.io import read as ase_read

import os
import json
import sys
sys.path.append("../py_src/")

import hydrogen_tracing
import config_plot

import re

In [ ]:
is_rutile = True
post_23_res = True

In [ ]:
rutile_dirs = [
    "rutile-001-nd-0",
    "rutile-011-nd-0",
    "rutile-100-nd-0",
    "rutile-110-nd-0",
]

anatase_dirs = [
    "anatase-100-nd-0",
    "anatase-101-nd-0", 
    "anatase-110-nd-0", 
]

if is_rutile:
    target_dirs = rutile_dirs
    savestr = 'rutile'
else:
    target_dirs = anatase_dirs
    savestr = 'anatase'

In [ ]:
all_counts = []
all_config_codes = []
all_hop_lists = []

all_bins = []
bin_maxes = []
bin_mins = []

total_min = -9
total_max = 0

for dirname in target_dirs:
    if post_23_res:
        dirname = "../test_data/230109_short_traj_newres/" + dirname
    else:
        dirname = "../test_data/221223_short_traj_res/" + dirname

    with open(os.path.join(dirname, os.path.basename(dirname)+'_non_numpy.json'), 'r') as f:
            non_numpy_results = json.load(f)
            config_dict = non_numpy_results['config_dict']
            special_list = non_numpy_results['special_list']
            f.close()

    counts = np.loadtxt(os.path.join(dirname, os.path.basename(dirname)+'_counts.txt'))
    config_codes = np.loadtxt(os.path.join(dirname, os.path.basename(dirname)+'_config_codes.txt'))
    hop_list = np.loadtxt(os.path.join(dirname, os.path.basename(dirname)+'_hop_list.txt'))

    all_counts.append(np.asarray(counts, dtype=np.int16))
    all_config_codes.append(np.asarray(config_codes, dtype=np.int8))
    all_hop_lists.append(np.asarray(hop_list, dtype=np.int8))

    # Make bin counts from last hop_list
    all_bins.append(np.bincount(all_hop_lists[-1]-np.min(all_hop_lists[-1])))
    bin_mins.append(np.min(all_hop_lists[-1]))
    bin_maxes.append(np.max(all_hop_lists[-1]))

    total_min = np.min(bin_mins)
    total_max = np.max(bin_maxes)
        
total_min = -9

In [ ]:
buff_bins = np.zeros((len(target_dirs), total_max-total_min+1))
for ii_bin, bincount in enumerate(all_bins):
    buff_bins[ii_bin, bin_mins[ii_bin]-total_min:bin_mins[ii_bin]-total_min+len(bincount)] = bincount

all_bins = buff_bins
label_range = np.arange(total_min, total_max+1)

 ## Raw Plot

In [ ]:
%matplotlib inline

# -9, -8, -7, -6, -5 ... -1
if not post_23_res:
    # 221223
    labels = ["Exchange of H?", "Undef.", "H20+Ti->H+HOTi\nH+HOTi->H2O+Ti", "No change at site", "Surface Hopping", "Wrong Oxygen Observed", "H20 close to HOTi", "OTi gaining H", "OTi losing H"]
else:
    # 230109
    labels = ["Exchange of H?", "Undef.", "H20+Ti->H+HOTi\nH+HOTi->H2O+Ti", "Flyby", "Surface Hopping", "Flyby", "H20 close to HOTi", "OTi gaining H", "OTi losing H"]
labels += np.arange(0, np.max(total_max)+1).tolist()

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.set_title("Analysis of Ti hopping for H2O Splitting")
ax.set_xlabel('Error Codes \ Number of Hops')
ax.set_ylabel('# of occurences')

n_bars = len(target_dirs)
if n_bars % 2: # not even:
    offsets = (np.arange(1, n_bars+1) - np.ceil(n_bars/2.)) / n_bars
else:
    offsets = (np.arange(n_bars) + 0.5 - (n_bars/2)) / n_bars

for ii_dir in range(n_bars):
    hop_counts = all_bins[ii_dir]
    ax.bar(label_range + offsets[ii_dir], hop_counts, width=1./n_bars, label=target_dirs[ii_dir])

ax.set_xticks(
    label_range,
    labels,
    rotation=-45, ha="left"
)
ax.legend()
plt.show()

## Post analysis

In [ ]:
def find_0_index(labels):
    for ii_label in range(-1, (-len(labels))-1, -1):
        try:
            int(labels[ii_label])
        except:
            return ii_label

In [ ]:
# last_string = find_0_index(labels)
if post_23_res:
    oti_index = 8
    closecall_index = [3, 5, 6]
else:
    oti_index = 8
    closecall_index = 6

erase_mistakes = True
mist_max = 1

find_hopping = True

plot_bins = all_bins.copy()
if find_hopping:
    # Find cases where TiO to TiO dissociation happens
    plot_bins[:, oti_index] = np.min(plot_bins[:, oti_index-1:oti_index+1], axis=-1)
    plot_bins[:, oti_index-1] = 0

    # Remove close calls ("H20 close to HOTi") manually
    plot_bins[:, closecall_index] = 0

    labels = ["Exchange of H?", "Undef.", "proton transfer\ntype 2", "No change at site", "Surface Hopping", "Wrong Oxygen Observed", "H20 close to HOTi", "Deleted", "proton transfer\ntype 1"]
    labels += np.arange(0, np.max(total_max)+1).tolist()

if erase_mistakes:
    plot_bins[plot_bins < mist_max] = 0
    
is_zero = np.logical_and.reduce(plot_bins==0, axis=0)
print(is_zero)

## Pretty Plot

In [ ]:
number_dict = {"0": "zero", "1": "one", "2": "two", "3": "three", "4": "four", "5": "five"}
nz_labels = []

# Remove zero labels
for ii_nz in range(len(is_zero)):
    if not is_zero[ii_nz]:
        nz_labels.append(labels[ii_nz])

# Proper step labelling
for ii_label, label in enumerate(nz_labels):
    try:
        int_label = int(label)
        nz_labels[ii_label] = number_dict[str(int_label+1)] + "-step"
    except:
        pass

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.set_title("Analysis H2O Dissociation")
ax.set_xlabel('Process')
ax.set_ylabel('# of occurences')

n_nz = np.sum(np.logical_not(is_zero).astype(np.int8))
plot_ticks = np.arange(n_nz)*1.2 # for slight spacing

for ii_dir in range(n_bars):
    ax.bar(plot_ticks+offsets[ii_dir], plot_bins[ii_dir, np.logical_not(is_zero)], width=1./n_bars, label=target_dirs[ii_dir])

ax.set_xticks(
    plot_ticks,
    nz_labels,
    rotation=-45, ha="left"
)
ax.legend()

# ax.set_xlim((ax.get_xlim()[0]+1, ax.get_xlim()[1]))
# ax.set_xlim([-0.5, 4.5])

plt.tight_layout()
fig.savefig("prettybars_%s.pdf"%savestr, format="pdf")
plt.show()

In [ ]:
plot_bins = all_bins.copy()

plot_bins[:, label_range<0] = 0
is_zero = np.logical_and.reduce(plot_bins==0, axis=0)

In [ ]:
print(plot_ticks, is_zero)

In [ ]:
nz_labels = []
# Remove zero labels
for ii_nz in range(len(is_zero)):
    if not is_zero[ii_nz]:
        nz_labels.append(labels[ii_nz])

# Proper step labelling
for ii_label, label in enumerate(nz_labels):
    try:
        int_label = int(label)
        nz_labels[ii_label] = number_dict[str(int_label+1)] + "-step"
    except:
        pass

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.set_title("Analysis H2O Dissociation")
ax.set_xlabel('Process')
ax.set_ylabel('# of occurences')

n_nz = np.sum(np.logical_not(is_zero).astype(np.int8))
plot_ticks = np.arange(n_nz)*1.2 # for slight spacing

for ii_dir in range(n_bars):
    ax.bar(plot_ticks+offsets[ii_dir], plot_bins[ii_dir, np.logical_not(is_zero)], width=1./n_bars, label=target_dirs[ii_dir])

ax.set_xticks(
    plot_ticks,
    nz_labels,
    rotation=-45, ha="left"
)
ax.legend()

# ax.set_xlim((ax.get_xlim()[0]+1, ax.get_xlim()[1]))
# ax.set_xlim([-0.5, 4.5])

plt.tight_layout()
fig.savefig("onlyhops_%s.pdf"%savestr, format="pdf")
plt.show()